In [3]:
# this cell combines two json files into one dataframe after all the preprocessing steps

# fucntion for date conversion of openweather data
def convert_date(ts):
    import pandas as pd
    ts=pd.to_datetime(ts, unit='s')
    year=ts.year
    month=ts.month
    day=ts.day
    hour=ts.hour
    return [year, month, day, hour]

# combining the two json files into one dataframe for a particular zone
def combine_data(zone):
        import pandas as pd
        import json

        # reading openweather data
        with open(f"openweather/air_pollution{zone}.json") as f:
                openweather = json.load(f)

        # getting longitude and latitude
        longitude = openweather['coord']['lon']
        latitude = openweather['coord']['lat']

        # converting openweather data to dataframe
        arr=[convert_date(openweather['list'][i]['dt'])+[zone, longitude,latitude,openweather['list'][0]['main']['aqi']]+
                list(openweather['list'][i]['components'].values()) for i in range(len(openweather['list']))]
        openweather = pd.DataFrame(arr, columns=['year','month','day','hour','zone',
                'longitude','latitude','aqi','co','no','no2','o3','so2','pm2_5','pm10','nh3'])


        # reading meteo data
        with open(f"meteo/temperature{zone}.json") as f:
                meteo = json.load(f)
        
        arr=[convert_date(pd.to_datetime(meteo['hourly']['time'][i]))+[longitude,latitude,
                meteo['hourly']['temperature_2m'][i], meteo['hourly']['dewpoint_2m'][i]] for i in range(len(meteo['hourly']['time']))]
        meteo = pd.DataFrame(arr, columns=['year','month','day','hour','longitude','latitude','temperature','dewpt'])
        
        # # reading weatherbit data
        # with open(f"weatherbit/temperature{zone}.json") as f:
        #         weatherbit = json.load(f)

        # arr=[convert_date(weatherbit['data'][i]['ts'])+[longitude,latitude]+[weatherbit['data'][i]['app_temp']
        #         ,weatherbit['data'][i]['dewpt'],weatherbit['data'][i]['weather']['description']] for i in range(len(weatherbit['data']))]
        # weatherbit = pd.DataFrame(arr, columns=['year','month','day','hour','longitude','latitude','temperature','dewpt','condition'])

        # merging openweather and weatherbit data
        return pd.merge(openweather, meteo, on=['year','month','day','hour','longitude','latitude']).drop([8734])

In [4]:
combine_data(1).to_csv('data.csv', index=False)

In [5]:
# concatenating all the dataframes into one
import pandas as pd
pd.concat([combine_data(i) for i in range(1, 91)]).sort_index(kind='merge').to_csv('combined_data.csv', index=False)

In [6]:
 '''
 Following data is missing from the combined data:
    1. 2021-01-27 00:01:00 - 2021-01-27 23:59:59 ~ 23 records
    2. 2021-01-28 00:00:00 - 2021-01-28 00:59:59 ~ 1 record
    3. 2021-12-31 22:00:00 - 2021-12-31 22:59:59 ~ 1 record

    Total missing records per zone: 25
    Total missing records for 90 zones: 2250
    
    Total records in combined data: 1,44,250
 '''

'\nFollowing data is missing from the combined data:\n   1. 2021-01-27 00:01:00 - 2021-01-27 23:59:59 ~ 23 records\n   2. 2021-01-28 00:00:00 - 2021-01-28 00:59:59 ~ 1 record\n   3. 2021-12-31 22:00:00 - 2021-12-31 22:59:59 ~ 1 record\n\n   Total missing records per zone: 25\n   Total missing records for 90 zones: 2250\n   \n   Total records in combined data: 1,44,250\n'